In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import getpass
pd.options.mode.chained_assignment = None

In [3]:
# Ruta para pcs
user = getpass.getuser() # Capturo el usuario local / Ejemplo "Llan_", este es el nombre de mi usuario que inicie sesion en la pc

if   user== "USUARIO": # laptop @hugoferquiroz
     work = Path(r"D:\\Trabajo\Brecha-PRITE")
     data = Path(r"D:\\OneDrive\\Bases de datos\\Minedu compartido")
elif user== "hugo_": # laptop @hugoferquiroz
     work = Path(r"C:\\Users\\hugo_\\OneDrive - Ministerio de Educación\\MINEDU_2022\\RACIO")
     data = Path(r"C:\\OneDrive\\Bases de datos\\Minedu compartido")

# Construcción de la base de datos

## Matricula

In [33]:
# Importamos las bases de datos
df_mat_20 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2020',skiprows=3)
df_mat_21 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2021',skiprows=3)
df_mat_22 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2022',skiprows=3)

In [34]:
def clean_matricula(df, año):
    # Cambiar los nombres
    df.rename(columns={0:'Menor a 12 meses',
                        1:'Entre 12 meses a 1 año',
                        2:'Entre 1 año a 2 años',
                        3:'Entre 2 años a 3 años',
                        4:'Entre 3 años a 4 años',
                        5:'Entre 4 años a 5 años',
                        'TOTAL':'Matricula reportada'}, inplace=True) 
    # Eliminar columnas innecesarias    
    if 'ERROR EN REGISTRO' in  df.columns.to_list():
        df.drop(columns=['ERROR EN REGISTRO'], inplace=True)
    # Verificar la longitud del codigo modular
    df['COD_MOD'] = df['COD_MOD'].apply(lambda x: x.zfill(7))
    # Eliminar la ultima fila = fila del total
    df.drop(df[df['COD_MOD'].str.find('TOTAL')!=-1].index,inplace=True)
    # Completar los missing de la matricula con 0
    df.fillna(0, inplace=True)
    # Agregar el año de la base
    df['Año'] = año
    return df

In [35]:
# Limpio las bases
df_mat_2020 = clean_matricula(df_mat_20,2020)
df_mat_2021 = clean_matricula(df_mat_21,2021)
df_mat_2022 = clean_matricula(df_mat_22,2022)

In [36]:
# Consolido las bases
matricula = pd.concat([df_mat_2020,df_mat_2021])
matricula = pd.concat([matricula,df_mat_2022])

In [37]:
# Doy formato a las columnas
    ## Genero las columnas en el orden que quiero
columnas = matricula.columns.to_list()
columnas_matricula = [columnas[0]] + [columnas[-3]] + columnas[1:5] + columnas[-2:] + [columnas[-4]]
columnas_matricula
    ## Cambio el orden del DataFrame
matricula = matricula[columnas_matricula]
    ## Imputo 0 a los missing\
matricula.fillna(0,inplace=True)

In [38]:
matricula.sample(5)

,COD_MOD,Año,Menor a 12 meses,Entre 12 meses a 1 año,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 3 años a 4 años,Entre 4 años a 5 años,Matricula reportada
41,1407543,2022,7.0,11.0,17.0,0.0,0.0,0.0,35
35,1366350,2021,5.0,6.0,13.0,2.0,0.0,0.0,26
91,1766526,2020,0.0,0.0,4.0,0.0,0.0,0.0,4
4,0901801,2021,9.0,18.0,26.0,0.0,0.0,0.0,53
27,1217728,2021,19.0,34.0,59.0,0.0,0.0,0.0,112


Evolución de la matricula

In [10]:
matricula_año = matricula.groupby('Año', as_index=False)['Matricula reportada'].agg('sum')

In [11]:
matricula_año.to_excel(work/r'tablas\\Evolución de matricula.xlsx')

## Personal

NEXUS al 4 de setiembre del 2022

In [12]:
# Importo la base de datos
nexus = pd.read_stata(data/r'Nexus\\2022\\nexus_35sira.dta')

c:\Users\USUARIO\anaconda3\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [13]:
# Selecciono los ssee identificados como PRITE
ssee_prite = matricula[['COD_MOD']]
ssee_prite.drop_duplicates(inplace=True) 
personal = pd.merge(left= ssee_prite, right=nexus, left_on='COD_MOD', right_on='codmod', how='left', validate='1:m',indicator=True)

In [14]:
# Funcion para identificar los duplicados por orden de prioridad
def plaza_unica(dataframe,codigo_modular='codmod'):
    print('--------------------------------------------------------------')
    print('Transformación de variables')
    dataframe['codmod'] = dataframe['codmod'].apply(str)
    dataframe.rename(columns={'codmod':'cod_mod'},inplace=True)
    dataframe['cod_mod'] = dataframe['cod_mod'].apply(lambda x: x.zfill(7))
    print('1. Verificar cantidad de digitos código modular')
    
    dataframe = dataframe[(dataframe.sitlab != "B") & (dataframe.sitlab != "P")]
    print('2. Eliminar plazas bloqueadas y previstas')
    
    dataframe['prior_tipo'] = 1
    dataframe.loc[dataframe['tiporegistro']== 'EVENTUAL', 'prior_tipo'] = 2
    dataframe.loc[dataframe['tiporegistro']== 'PROYECTO', 'prior_tipo'] = 3
    dataframe.loc[dataframe['tiporegistro']== 'CUADRO DE HORAS', 'prior_tipo'] = 4
    dataframe.loc[dataframe['tiporegistro'].str.find('REEMPLAZO')!=-1, 'prior_tipo'] = 5
    print('3. Priorizacion del tipo de plaza')

    dataframe['prior_sitlab'] = 1
    dataframe.loc[(dataframe['sitlab']== 'F') | (dataframe['sitlab']== 'D') 
                 | (dataframe['sitlab']== 'E') | (dataframe['sitlab']== 'T'), 'prior_sitlab'] = 2
    dataframe.loc[(dataframe['sitlab']== 'C') | (dataframe['sitlab']== 'V'), 'prior_sitlab'] = 3
    print('4. Priorizacion de la situación laboral')

    dataframe['soplaza'] = 0
    dataframe.loc[((dataframe['estplaza'].str.find("SG")==-1) 
                  & (dataframe['estplaza'].str.find("CG")==-1) 
                  & (dataframe['estplaza'].str.find("ABAND")==-1)), 'soplaza'] = 1
    print('5. Identificación plaza con goce, sin goce')
    
    dataframe['sestpla'] = 0
    dataframe.loc[dataframe['estplaza']== 'ACTIV', 'sestpla'] = 1
    print('6. Identificación estado de la plaza')
    
    dataframe.loc[dataframe['jornlab']=='', 'jornlab'] = 99
    dataframe = dataframe.sort_values(by='jornlab', ascending=False)
    dataframe['sjornlb'] = dataframe['jornlab'].rank(method='dense', ascending=False)
    print('7. Ordenamiento descendente de la jornada laboral')

    print('--------------------------------------------------------------')
    dataframe = dataframe.sort_values(['descreg', 'nombreooii', 'cod_mod', 'codplaza', 'prior_tipo', 
                          'prior_sitlab', 'soplaza', 'sestpla', 'sjornlb'])
    dataframe['tipo_fin'] = dataframe.groupby(['cod_mod', 'codplaza']).cumcount()
    dataframe['tipo_fin'] = dataframe['tipo_fin'] + 1
    print('Identificación plaza única')
    print(dataframe.groupby(['tipo_fin'])['tipo_fin'].count())
    print('Plaza original tienen la etiqueta igual a 1')
    return dataframe


In [15]:
# Identifico los duplicados
personal = plaza_unica(personal)

--------------------------------------------------------------
Transformación de variables
1. Verificar cantidad de digitos código modular
2. Eliminar plazas bloqueadas y previstas
3. Priorizacion del tipo de plaza
4. Priorizacion de la situación laboral
5. Identificación plaza con goce, sin goce
6. Identificación estado de la plaza
7. Ordenamiento descendente de la jornada laboral
--------------------------------------------------------------
Identificación plaza única
tipo_fin
1    519
2     28
3      1
4      1
Name: tipo_fin, dtype: int64
Plaza original tienen la etiqueta igual a 1


In [16]:
# Elimino los duplicados
personal = personal.loc[(personal['tipo_fin']==1) | (personal['_merge']=='left_only')]

Personal por regimen laboral

In [17]:
pd.crosstab(personal['descargo'],personal['desley'],margins=True)

desley,D.L. 1153,D.L. 276,DL 1057,LEY 29944,LEY 30328,LEY 30493,All
descargo,,,,,,,
AUXILIAR DE EDUCACION,0,0,0,0,0,8,8
COORDINACION PEDAGOGICO,0,0,0,1,0,0,1
COORDINADOR ACADEMICO,0,0,0,5,0,0,5
DIRECTOR I.E.,0,0,0,33,0,0,33
DOCENTE COORDINADOR,0,0,0,0,1,0,1
FISIOTERAPEUTA,0,1,0,0,0,0,1
OFICINISTA,0,2,0,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,0,123,0,0,0,123
PROFESOR,0,0,0,121,146,0,267


In [18]:
# Exporto la tabla
(pd.crosstab(personal['descargo'],personal['desley'],margins=True)).to_excel(work/r'tablas\\personal por regimen laboral.xlsx')

Personal por tipo de trabajador

In [19]:
pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)

desctipotrab,ADMINISTRATIVO,CAS,DOCENTE,PROFESIONAL DE LA SALUD,All
descargo,,,,,
AUXILIAR DE EDUCACION,0,0,8,0,8
COORDINACION PEDAGOGICO,0,0,1,0,1
COORDINADOR ACADEMICO,0,0,5,0,5
DIRECTOR I.E.,0,0,33,0,33
DOCENTE COORDINADOR,0,0,1,0,1
FISIOTERAPEUTA,1,0,0,0,1
OFICINISTA,2,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,123,0,0,123
PROFESOR,0,0,267,0,267


In [20]:
(pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)).to_excel(work/r'tablas\\personal por tipo de trabajador.xlsx')

In [21]:
# Homogenizo los perfiles
    ## Docentes
personal['Profesor'] = 0
personal.loc[(personal['descargo']=='PROFESOR') | 
             (personal['descargo']=='PROFESOR (FUNCIONES DE DIRECTOR)') | 
             (personal['descargo']=='PROFESOR COORDINADOR') |
             (personal['descargo']=='COORDINACION PEDAGOGICO') |
             (personal['descargo']=='AUXILIAR DE EDUCACION') | 
             (personal['descargo']=='DOCENTE COORDINADOR') |        
             (personal['descargo']=='COORDINADOR ACADEMICO') ,'Profesor'] =1 
    ## Director
personal['Director'] = 0
personal.loc[(personal['descargo']=='DIRECTOR I.E.') ,'Director'] =1 
    ## Tecnologos medicos
personal['Tecnologo medico'] = 0
personal.loc[(personal['descargo']=='PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA') | 
             (personal['descargo']=='TERAPISTA') | 
             (personal['descargo']=='FISIOTERAPEUTA'),'Tecnologo medico'] =1     
    ## Psicologos
personal['Psicologos'] = 0
personal.loc[(personal['descargo'].str.find('PSICOLOGO')!=-1) ,'Psicologos'] =1 
    ## Personal administrativo
personal['Administrativos'] = 0
personal.loc[(personal['descargo']=='TRABAJADOR DE SERVICIO') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO II') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO III') | 
             (personal['descargo']=='OFICINISTA') |                              
             (personal['descargo']=='TECNICO ADMINISTRATIVO'),'Administrativos'] =1

In [22]:
# Colapso el DataFrame a nivel de servicio educativo
trabajadores = personal.groupby('COD_MOD', as_index=False).agg('sum')
# Doy formato a la base
columnas = trabajadores.columns.tolist()
columnas_nexus = [columnas[0]] + columnas[-5:]
trabajadores = trabajadores[columnas_nexus]
trabajadores.sample(5)

,COD_MOD,Profesor,Director,Tecnologo medico,Psicologos,Administrativos
55,1566322,2,0,1,0,0
67,1702828,2,0,1,0,0
11,1031749,3,1,1,0,0
85,1739069,2,0,1,0,0
21,1113505,3,0,1,0,0


## Datos del padron web
Padron web descargado de la [pagina web](http://escale.minedu.gob.pe/uee/-/document_library_display/GMv7/view/958881) de escale con fecha 17/09/2022 y complemetado con información de las asignaciones temporales y unidades ejecutoras de la Unidad de Presupuesto

In [23]:
# Importo el padron de la Unidad de Presupuesto 
df_padron = pd.read_stata(data/r'Padron GG1.dta')
df_padron = df_padron.loc[df_padron['niv_mod']=='E0']

c:\Users\USUARIO\anaconda3\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [24]:
# Identifico a los servicios PRITE
padron = pd.merge(left= ssee_prite, right=df_padron, left_on='COD_MOD', right_on='cod_mod', how='left', validate='1:m',indicator=True)
# Filtro las columnas relevantes
datos_padron = ['COD_MOD','cen_edu','d_gestion','d_ges_dep','ubigeo','d_dpto','d_prov', 'd_dist','d_region','codooii', 'd_dreugel','nlat_ie','nlong_ie',
                'vraem_upp_2022','tipie_upp_2022','bilin_upp_2022','rural_upp_2022','fron_upp_2022','region','pliego','codue', 'unidadejecutora', 'nombentidad']
padron_web = padron[datos_padron]

In [25]:
padron_web.sample(5)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,vraem_upp_2022,tipie_upp_2022,bilin_upp_2022,rural_upp_2022,fron_upp_2022,region,pliego,codue,unidadejecutora,nombentidad
83,1747112,PRITE ANDAHUAYLAS,P£blica de gesti¢n directa,Sector Educaci¢n,030201,APURIMAC,ANDAHUAYLAS,ANDAHUAYLAS,DRE APURIMAC,030002,...,No Vraem,Completo,No bilingue,Urbano,No frontera,APURIMAC,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,442301,301. EDUCACION CHANKA,REGION APURIMAC-EDUCACION CHANKA
51,1538867,PRITE SAVIA PERU,P£blica de gesti¢n privada,Convenio con Sector Educaci¢n,200703,PIURA,TALARA,LA BREA,DRE PIURA,200011,...,No Vraem,Completo,No bilingue,Urbano,No frontera,PIURA,457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA,457306,306. EDUCACION UGEL DE TALARA,GOB. REG. DE PIURA - EDUCACION UGEL DE TALARA
2,0780874,PRITE LUIS AQUILES GUERRA,P£blica de gesti¢n directa,Sector Educaci¢n,150112,LIMA,LIMA,INDEPENDENCIA,DRE LIMA METROPOLITANA,150103,...,No Vraem,Completo,No bilingue,Urbano,No frontera,LIMA METROPOLITANA,10. LIMA METROPOLITANA,102,002. USE 02 SAN MARTIN DE PORRAS,USE 02 SAN MARTIN DE PORRAS
64,1702828,PRITE JUAN PABLO II,P£blica de gesti¢n directa,Sector Educaci¢n,120808,JUNIN,YAULI,SANTA ROSA DE SACCO,DRE JUNIN,120009,...,No Vraem,Completo,No bilingue,Urbano,No frontera,JUNIN,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,450308,308. EDUCACION YAULI - LA OROYA,GOB. REG. DE JUNIN - EDUCACION YAULI- LA OROYA
57,1600477,PRITE MARIA DEL ROSARIO,P£blica de gesti¢n directa,Sector Educaci¢n,250107,UCAYALI,CORONEL PORTILLO,MANANTAY,DRE UCAYALI,250001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,UCAYALI,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,462303,303. EDUCACION CORONEL PORTILLO,GOB. REG. DE UCAYALI - EDUCACION CORONEL PORTILLO


# Calculo de la brecha

## Consolidación de la base

In [45]:
df = pd.merge(left=padron_web, right=df_mat_2022, on='COD_MOD')
df = pd.merge(left=df, right=trabajadores, on='COD_MOD')
df.sample(3)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 3 años a 4 años,Matricula reportada,Año,Profesor,Director,Tecnologo medico,Psicologos,Administrativos
13,1036417,PRITE CHILCA,P£blica de gesti¢n directa,Sector Educaci¢n,120107,JUNIN,HUANCAYO,CHILCA,DRE JUNIN,120001,...,26,0.0,0.0,71,2022,4,1,1,1,0
9,1025881,PRITE PUNO,P£blica de gesti¢n directa,Sector Educaci¢n,210101,PUNO,PUNO,PUNO,DRE PUNO,210001,...,40,0.0,0.0,90,2022,4,1,2,0,0
67,1704766,PRITE SAN MIGUEL,P£blica de gesti¢n directa,Sector Educaci¢n,080601,CUSCO,CANCHIS,SICUANI,DRE CUSCO,080006,...,16,0.0,0.0,32,2022,2,0,1,0,0


## Calculo los ratios de asignación

De acuerdo al [RVM 188-2019](https://cdn.www.gob.pe/uploads/document/file/348345/RVM_N__188-2019-MINEDU.pdf) como mínimo se asigna 1 personal docente, psicólogo y técnologo médico por cada 12 alumnos menores de 3 años. 

Los niños y niñas de 3 años de edad con discapacidad severa o multidiscapacidad pueden permanecer en el PRITE hasta los 5 años de edad, solo si se encuentran en una zona rural y/o bilingue o si en su jurisdicción no se cuanta con un CEBE

In [58]:
df['Personal segun norma'] = df['Matricula reportada']/12
df['Personal segun norma'] = df['Personal segun norma'].astype(int)
df['Director norma'] = 1

In [84]:
# Requerimientos
personal = ['Profesor','Tecnologo medico','Psicologos']

for prof in personal:
    df[f'{prof} requerimiento'] = np.where(df[f'{prof}']<=df['Personal segun norma'],df['Personal segun norma']-df[f'{prof}'],0)
    df[f'{prof} excedente'] = np.where(df[f'{prof}']>=df['Personal segun norma'],df[f'{prof}']-df['Personal segun norma'],0)

In [85]:
df[['Personal segun norma','Profesor','Profesor requerimiento','Profesor excedente']].sample(5)

,Personal segun norma,Profesor,Profesor requerimiento,Profesor excedente
1,2,4,0,2
102,3,5,0,2
65,1,1,0,0
31,1,1,0,0
9,7,4,3,0


In [114]:
def table_costo(data_frame):
    # Calculo el costo
    w_psico, w_profe,w_tec_medico = 2500 , 2500, 2000 
    data_frame['Costo-Profesor'] = data_frame['Brecha-Profesor']*w_profe
    data_frame['Costo-Tecnologo medico'] = data_frame['Brecha-Tecnologo medico']*w_tec_medico
    data_frame['Costo-Psicologo'] = data_frame['Brecha-Psicologo']*w_profe

    # Doy formato a la tabla
    data_frame.loc['Total'] = data_frame.sum()
    data_frame_resumen = data_frame.style.format({'Costo-Profesor':'S/{:20,.0f}','Costo-Tecnologo medico':'S/{:20,.0f}','Costo-Psicologo':'S/{:20,.0f}' })
    return data_frame_resumen

## Brecha sin optimizar

En este escenario no se usan las plazas excedentes para cubrir los requerimientos identificados por lo que la brecha es equivalente a los requerimientos identificados 

In [102]:
# Colapso la brecha por region
sr = df.groupby(['region'])['Profesor','Profesor requerimiento','Profesor excedente',
                            'Tecnologo medico','Tecnologo medico requerimiento','Tecnologo medico excedente',
                            'Psicologos','Psicologos requerimiento','Psicologos excedente'].sum()
sr = sr[['Profesor requerimiento','Tecnologo medico requerimiento','Psicologos requerimiento']]
sr.rename(columns={'Profesor requerimiento': 'Brecha-Profesor',
                    'Tecnologo medico requerimiento': 'Brecha-Tecnologo medico',
                    'Psicologos requerimiento': 'Brecha-Psicologo'},inplace=True)                           

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_23480\690981780.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sr = df.groupby(['region'])['Profesor','Profesor requerimiento','Profesor excedente',


In [116]:
sr_region = table_costo(sr)
sr_region

,Brecha-Profesor,Brecha-Tecnologo medico,Brecha-Psicologo,Costo-Profesor,Costo-Tecnologo medico,Costo-Psicologo
region,,,,,,
AMAZONAS,1,1,2,"S/ 2,500","S/ 2,000","S/ 5,000"
ANCASH,1,13,16,"S/ 2,500","S/ 26,000","S/ 40,000"
APURIMAC,4,1,9,"S/ 10,000","S/ 2,000","S/ 22,500"
AREQUIPA,2,7,12,"S/ 5,000","S/ 14,000","S/ 30,000"
AYACUCHO,0,4,5,S/ 0,"S/ 8,000","S/ 12,500"
CAJAMARCA,1,2,8,"S/ 2,500","S/ 4,000","S/ 20,000"
CALLAO,1,10,13,"S/ 2,500","S/ 20,000","S/ 32,500"
CUSCO,10,14,35,"S/ 25,000","S/ 28,000","S/ 87,500"
HUANCAVELICA,1,2,5,"S/ 2,500","S/ 4,000","S/ 12,500"


## Brecha optimizada 

En este escenario se usan las plazas excedentes para cubrir los requerimientos identificados siempre y cuando estos excedentes sean reubicados dentro de la region

In [ ]:
# Calculo la diferencia por region

# Colapso esa diferencia por region

# Costo 

# Formato 

# Exporto

In [ ]:
# Colapso la brecha por region
sr = df.groupby(['region'])['Profesor','Profesor requerimiento','Profesor excedente',
                            'Tecnologo medico','Tecnologo medico requerimiento','Tecnologo medico excedente',
                            'Psicologos','Psicologos requerimiento','Psicologos excedente'].sum()
sr = sr[['Profesor requerimiento','Tecnologo medico requerimiento','Psicologos requerimiento']]
sr.rename(columns={'Profesor requerimiento': 'Brecha-Profesor',
                    'Tecnologo medico requerimiento': 'Brecha-Tecnologo medico',
                    'Psicologos requerimiento': 'Brecha-Psicologo'},inplace=True)                           

# Priorizacion de recursos